# Lattice Dynamics and Density Functional Perturbation Theory - Harmonic Level

The script 'thermodyanmics.py' is a comprehensive module for lattice dynamics based on harmonic and quasi-harmonic level of theory. For simplicity, this tutorial is focused only on harmonic phonons. For quasi-harmonic part, please refer to 'thermodynamics-QHA'.

**Classes used in this tutorial are defined in 'CRYSTALpytools.thermodynamics'. Documentations and source codes are attached at the bottom of this page.**

## Tests
3 tests are performed. Two are based on $\Gamma$ point calculations and the other one is based on phonon dispersion calculations.

1. Thermodynamic properties at room temperature (298.15K), 0 pressure - For comparison with CRYSTAL17 output.  
1. Thermodynamic properties at room temperature (298.15K), 0 pressure - Converted to Phonopy FORCE_CONSTANTS file.  
2. Thermodynamic properties from 0K to 300K, with the interval of 20K and from 0 GPa to 0.5 GPa with the interval of - To illustrate the structure of output file and key information. 

**N.B.** It is somehow meanless to consider pressure & Gibbs free energy at Harmonic level, since the volume is assumed as constant. $G(T, p)=F(T)+pV$.

### Room temperature (298.15K), 0 pressure thermodyanmics
This part is used for comparison with CRYSTAL17 output file, in order to illustrate the equivalence of methods adopted here and the method of CRYSTAL17. $\Gamma$ point calculation of Form I paracetamol is used ('freqf1-r0.out' in 'data/'). Reference data with the same environmental conditions can be found in the same file. No output file is generated. 

The 'pressure' specified is the value given in the same output file. For comparison only.

In [1]:
from CRYSTALpytools import thermodynamics
from CRYSTALpytools.thermodynamics import Harmonic

rm_condition = Harmonic(filename=None, autocalc=False).from_file(
    'data/freqf1-r0.out', scelphono=[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
)
rm_condition.thermodynamics(temperature=[298.15], pressure=[0.10132500E-3])


# Note: All attributes are lists.
print('DFT total energy (EL) = ', rm_condition.edft, 'KJ/mol')
print('Helmholtz free energy (EL+E0+ET-TS) = ', rm_condition.helmholtz[0, 0], ' KJ/mol')
print('Gibbs free energy (EL+E0+ET+pV-TS) = ', rm_condition.gibbs[0, 0, 0], ' KJ/mol')
print('Zero-point energy (E0) = ', rm_condition.zp_energy[0], ' KJ/mol')
print('Vibrational contribution to interla energy - E0 (ET) = ', rm_condition.u_vib[0, 0] - rm_condition.zp_energy[0], ' KJ/mol')
print('Entropy*Temperature (TS) = ', rm_condition.entropy[0, 0] * 298.15, ' J/mol')
print('Heat capacity = ', rm_condition.c_v[0, 0], ' J/mol*K')
print('Entropy = ', rm_condition.entropy[0, 0], ' J/mol*K')

DFT total energy (EL) =  -5402456.254965135 KJ/mol
Helmholtz free energy (EL+E0+ET-TS) =  -5400800.797467394  KJ/mol
Gibbs free energy (EL+E0+ET+pV-TS) =  -5400800.7514409665  KJ/mol
Zero-point energy (E0) =  1759.1003499056465  KJ/mol
Vibrational contribution to interla energy - E0 (ET) =  107.20366839971416  KJ/mol
Entropy*Temperature (TS) =  210846.5205645217  J/mol
Heat capacity =  665.5752785152763  J/mol*K
Entropy =  707.1826951686121  J/mol*K


References from 'freqf1-r0.out' (line 11703~11738) are attached below. Here are several findings:

1. Parameters derived from vibrational frequencies shows good agreement with reference data, with the error at the level of 10^-4 KJ/mol. The choice of physical constants and different unit conversion coefficients might lead to the discrepancy, where the physical constants play a major role (See below).  
2. The difference in DFT total energy lies in different coefficients for unit conversion. In CRYSTAL17 case, 1 Hartree = 2625.5 KJ/mol. In the current case, 1 Hartree = 2625.500256 KJ/mol.  
3. The PV term is neglected at room condition. Helmholtz free energy is the focus of this tutorial since the Harmonic class was initially developed for QHA fittings. To calculate Gibbs free energy at HA level, use this equation: $G(T,p) = F(T) + pV$

```
 *******************************************************************************

 HARMONIC VIBRATIONAL CONTRIBUTIONS TO THERMODYNAMIC FUNCTIONS AT GIVEN
 TEMPERATURE AND PRESSURE:

 (EL = ELECTRONIC ENERGY
  E0 = ZERO-POINT ENERGY
  ET = THERMAL CONTRIBUTION TO THE VIBRATIONAL ENERGY
  PV = PRESSURE * VOLUME
  TS = TEMPERATURE * ENTROPY)

                          AU/CELL             EV/CELL                 KJ/MOL
 EL            :   -2057.686915559598  -55992.507576455653    -5402456.23545757
 E0            :       0.670005954735      18.231788914567        1759.10038625


 *******************************************************************************

 THERMODYNAMIC FUNCTIONS WITH VIBRATIONAL CONTRIBUTIONS

 AT (T =  298.15 K, P =   0.10132500E+00 MPA):

                          AU/CELL             EV/CELL                 KJ/MOL
 ET            :       0.040831798552       1.111089725297         107.20387199
 PV            :       0.000017530544       0.000477030356           0.04602644
 TS            :       0.080307302905       2.185272809177         210.84679406
 ET+PV-TS      :      -0.039457973810      -1.073706053524        -103.59689564
 EL+E0+ET+PV-TS:   -2057.056367578673  -55975.349493594607    -5400800.73196695

 OTHER THERMODYNAMIC FUNCTIONS:

                      mHARTREE/(CELL*K)     mEV/(CELL*K)              J/(MOL*K)
 ENTROPY       :       0.269352013769       7.329440916241         707.18361249
 HEAT CAPACITY :       0.253504784422       6.898215882635         665.57671770

 *******************************************************************************
```

### Conversion to Phonopy FORCE\_CONSTANTS file
The numerical Hessian is printed out in a formatted 'HESSFREQ.DAT' file by CRYSTAL, which can be converted into a Phonopy FORCE\_CONSTANTS file by the `thermodynamics.Phonopy.write_force_constants()` method.

The 'data/freqf1-r0.HESSFREQ' file is used as an example. Phonopy required.

In [1]:
from CRYSTALpytools.thermodynamics import Phonopy

Phonopy.write_force_constants(hessfile='data/freqf1-r0.HESSFREQ', phonopyfile='data/FORCE_CONSTANTS')
! cd data; phonopy --crystal --qpoints='0 0 0' -c freqf1-r0.out --dim='1 1 1' --readfc

        _
  _ __ | |__   ___  _ __   ___   _ __  _   _
 | '_ \| '_ \ / _ \| '_ \ / _ \ | '_ \| | | |
 | |_) | | | | (_) | | | | (_) || |_) | |_| |
 | .__/|_| |_|\___/|_| |_|\___(_) .__/ \__, |
 |_|                            |_|    |___/
                                      2.19.1

Compiled with OpenMP support (max 256 threads).
Python version 3.9.16
Spglib version 2.0.2


Calculator interface: crystal
Crystal structure was read from "freqf1-r0.out".
Unit of length: angstrom
QPOINTS mode
Settings:
  Supercell: [1 1 1]
Spacegroup: P2_1/c (14)
Use -v option to watch primitive cell, unit cell, and supercell structures.

Force constants are read from "FORCE_CONSTANTS".
Force constants format was transformed to compact format.
Array shape of force constants: (80, 80, 3, 3)

Q-points that will be calculated at:
    [0. 0. 0.]

Summary of calculation was written in "phonopy.yaml".
                 _
   ___ _ __   __| |
  / _ \ '_ \ / _` |
 |  __/ | | | (_| |
  \___|_| |_|\__,_|



The phonon frequencies in THz at $\Gamma$ point are printed into the 'data/qpoints.yaml' and the geometry information is printed into 'data/phonopy.yaml'. Slight deviations are observed because CRYSTAL fully utilizes the symmetry which makes the first 3 translational modes to be 0.

### Multiple q points, Multiple temperatures & pressures
Class Harmonic can calculate the HA thermodynamics at a set of q points in order to sample various cell sizes. As a test case illustrating the effectiveness of this, a phonon dispersion calculation is performed based on the $\Gamma$ point frequency data of Form I paracetamol crystal. `SCELPHONO` is set to be 1 to save computation time.

The reference data is 'f1-disp.out' - Frequencies at $\Gamma$ are different from the 'f1freq-r0.out' case due to unknown reasons. In this case, nqpoint = 140, nmode = 240, natom = 80. The temperature range is set to be 0K ~ 300K, with intervals of 30K while the pressure range is set to be 0 GPa ~ 0.5 GPa, with intervals of 0.1 GPa. Since negative frequencies presents, warning messages will be printed. 

All the calculations are automatically executed. The output file is 'f1-disp-HA.dat'. Phonon dispersions are summed up, which, in the current case, is not necessary since the primitive cell is sufficiently large. This option is useful for smaller primitive cells.

In [2]:
import numpy as np

t = np.arange(0, 301, 30)
p = np.arange(0, 0.51, 0.1)
Harmonic(temperature=t, pressure=p,
         filename='data/f1-disp-HA.dat',
         autocalc=True).from_file('data/f1-disp.out',
                                  scelphono=[[1, 0, 0], [0, 1, 0], [0, 0, 1]])

/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/thermodynamics.py:600: UserWarning: Negative frequencies detected.
  Calculated thermodynamics might be inaccurate. Negative frequencies will be substituted by NaN.
  self = PhononBASE.clean_imaginary(self, threshold=imaginary_tol)
/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/thermodynamics.py:601: UserWarning: Overlap of q points is detected between q points  19 and  20
  self = PhononBASE.clean_q_overlap(self, threshold=q_overlap_tol)
/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/thermodynamics.py:601: UserWarning: Overlap of q points is detected between q points  39 and  40
  self = PhononBASE.clean_q_overlap(self, threshold=q_overlap_tol)
/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/thermodynamics.py:601: UserWarning: Overlap of q